# Artwork sequence prediction

## Load data

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

df_impressionist_sorted = pd.read_csv('impressionsist_sorted.csv')
impressionist_sorted_matrix = np.load('impressionist_sorted_matrix.npy')
df_impressionist_sorted.head()

,filename,title,style,genre,date,artist,imageUrl
0,44196.jpg,"La Hacienda Buenavista en Ponce, Puerto Rico",Impressionism,landscape,1840,francisco oller,https://uploads8.wikiart.org/images/francisco-...
1,50608.jpg,Woman and Child on a Bridge,Impressionism,genre painting,1848,honore daumier,https://uploads4.wikiart.org/images/honore-dau...
2,27526.jpg,On the Shore,Impressionism,genre painting,1853,honore daumier,https://uploads7.wikiart.org/images/honore-dau...
3,102063.jpg,Bathers,Impressionism,genre painting,1853,honore daumier,https://uploads4.wikiart.org/images/honore-dau...
4,66688.jpg,Study for the Self Portrait,Impressionism,self-portrait,1855,edgar degas,https://uploads6.wikiart.org/images/edgar-dega...


In [3]:
time = np.arange(impressionist_sorted_matrix.shape[0])

## Reset Tensorflow session

In [4]:
tf.keras.backend.clear_session()

## Split dataset

In [5]:
split_time = 4500

time_train = time[:split_time]
time_valid = time[split_time:]

X = impressionist_sorted_matrix

window_size = 20
batch_size = 256
shuffle_buffer_size = 1000

## Create models

In [6]:
from Prediction_model_feature import Prediction_model_feature

In [7]:
n_features = X.shape[1]
models = []

In [ ]:
from IPython.display import clear_output
import time

start_time = time.time()
for i in range(n_features):
    clear_output(wait=True)
    print("---------- Feature %s -------------" % (i))
    model_prediction = Prediction_model_feature(
        X=X[:, i],
        split_time=split_time,
        train_batch_size=batch_size, 
        val_batch_size=batch_size, 
        window_size=window_size, 
        shuffle_buffer=shuffle_buffer_size,
        name="feature " + str(i))
    model_prediction.define_model()
    model_prediction.train_model(epochs=20, lr=1e-6)
    models.append(model_prediction)
    
print("--- %s seconds ---" % (time.time() - start_time))

---------- Feature 33 -------------
Epoch 1/20
18/18 [==============================] - 7s 372ms/step - loss: 1.8105 - mae: 2.2765 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/20
18/18 [==============================] - 2s 89ms/step - loss: 0.4784 - mae: 0.8532 - val_loss: 0.3597 - val_mae: 0.7121
Epoch 3/20
18/18 [==============================] - 2s 86ms/step - loss: 0.3279 - mae: 0.6608 - val_loss: 0.2665 - val_mae: 0.5863
Epoch 4/20
18/18 [==============================] - 2s 89ms/step - loss: 0.2754 - mae: 0.5870 - val_loss: 0.2358 - val_mae: 0.5463
Epoch 5/20
18/18 [==============================] - 2s 88ms/step - loss: 0.2498 - mae: 0.5540 - val_loss: 0.2186 - val_mae: 0.5214
Epoch 6/20
18/18 [==============================] - 2s 87ms/step - loss: 0.2350 - mae: 0.5336 - val_loss: 0.2072 - val_mae: 0.5038
Epoch 7/20
18/18 [==============================] - 2s 88ms/step - loss: 0.2246 - mae: 0.5191 - val_loss: 0.1988 - val_mae: 0.4922
Epoch 8/20
18/18 [====================

In [ ]:
len(models)

In [ ]:
models[0].model.summary()

## Evaluate model

### Load decoder model

In [ ]:
from tensorflow.python.keras.models import load_model

In [ ]:
decoder_model = load_model('wasserstein_decoder.h5')
decoder_model.summary()

**Test decoder with example**

In [ ]:
p = decoder_model.predict(x_train[1].reshape((1,1,1,-1)))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(p[0][...,::-1])

### Predict features

**Sliced validation dataset**

In [ ]:
def validation_dataset(series, window_size, batch_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

**Predict features**

In [ ]:
prediction = []

for i in range(n_features):
    val_dataset = validation_dataset(x_valid[:,i], window_size, batch_size)
    for x, y in val_dataset.take(1):
        prediction_feature = models[i].model.predict(x)[0]
        prediction.append(prediction_feature)

In [ ]:
code = []
for p in prediction:
    code.append(prediction[0][0])

code = np.array(code)

In [ ]:
code = code.reshape((-1,))

**Decode code**

In [ ]:
p = decoder_model.predict(code.reshape((1,1,1,-1)))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(p[0])

In [ ]:
valid_decode = decoder_model.predict(x_valid[0].reshape((1,1,1,-1)))

In [ ]:
plt.imshow(valid_decode[0][...,::-1])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

cosine_distances(code, x_valid[0][...,::-1])

### Plot feature prediction

In [ ]:
from utils_plot import plot_series, plot_train_history, plot_prediction
import tensorflow as tf

In [ ]:
val_dataset = validation_dataset(x_valid[:,0], window_size, batch_size)
for x, y in val_dataset.take(1):
    prediction = models[0].model.predict(x)[0]
    plot = plot_prediction([x[0].numpy(), y[0].numpy(), prediction[0]] , 'Simple LSTM model')
    plot.show()

In [ ]:
val_dataset = validation_dataset(x_valid[:,20], window_size, batch_size)
for x, y in val_dataset.take(1):
    prediction = models[20].model.predict(x)[0]
    plot = plot_prediction([x[0].numpy(), y[0].numpy(), prediction[0]] , 'Simple LSTM model')
    plot.show()

**Predict x_valid**

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(batch_size).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
x = np.concatenate((x_train[:, 1], x_valid[:, 1]))

In [ ]:
rnn_forecast = model_forecast(model_prediction.model, x, window_size, batch_size)

In [ ]:
rnn_forecast = rnn_forecast[split_time-window_size+1:,-1,0]

In [ ]:
plot_series(time_valid, [rnn_forecast], label="rnn")